In [2]:
from project.survey import SurveyReader
from project.constants import DATA_DIR, STUDY_YEARS


survey = SurveyReader(DATA_DIR, include_years=STUDY_YEARS)
survey.read_files()

q_names = [
    # "P501",   # Last week were you employed?
    # "P510",   # Kind of work performed
    # "P5111",  # Sueldo
    # "P5112",  # Salario
    # "P5113",  # Comision
    # "P5114",  # Destajo
    # "P5115",  # Subvencion
    # "P5116",  # Honorarios
    # "P51112",  # Does NOT receive any kind of payment (1) 
    # "P523",   # Frequency of payment
    # "P524A1",  # Total income. 999999 missing value
    # "P5295A",  # Frequency of health payments
    # "P5295B",  # Amount paid in health payments. 999999 missing value
    # "P5295C",  # Does not know
    # "P524A1",  # Total income, 999999 missing value
    # "P524A2",  # 1 means "Don't know" for P524A1 
    # "P524B1",  # Legal discount, 999999 missing value
    # "P524B2",  # 1 means "Don't know" for P524B1
    # "P524C1",  # Tax, 999999 missing value
    # "P524C2",  # 1 means "Don't know" for P524C1
    # "P524D1",  # Other discount, 999999 missing value
    # "P524D2",  # 1 means "Don't know" for P524D1
    # "P524E1",  # Liquid income, 999999 missing value
    # "P524E2",  # 1 means "Don't know" for P524E1
    # "P5295A",
    # "P5295B",
    # "P5295C",
    "P530A",   # Net income last period, 999999 missing value
    "P530B",   # 1 means "Don't know" for P530A
    # "P536",    # Estimate of consumption products, 999999 missing value
    # "P538A1",  # Total income last month, 999999 missing value
    # "P538A2",  # 1 means "Don't know" for P538A1
    # "P538B1",  # Legal discount last month, 999999 missing value
    # "P538B2",  # 1 means "Don't know" for P538B1
    # "P538C1",  # Tac last month, 999999 missing value
    # "P538C2",  # 1 means "Don't know" for P538C1
    # "P538D1",  # Other discount last month, 999999 missing value
    # "P538D2",  # 1 means "Don't know" for P538D1
    
]

In [5]:
income_qs = survey.data_columns("05", q_names)#.sample(5000)
unique_person = len(income_qs["PERSON_KEY"].unique())
unique_house = len(income_qs["HOUSE_KEY"].unique())
print(unique_house, unique_person)

34490 91315


In [3]:
from project.income import compute_poverty_thresholds, MONTHLY_INCOME_COLUMN
# Replace 999999 with 0 (missing value)
income_qs[MONTHLY_INCOME_COLUMN] = income_qs[MONTHLY_INCOME_COLUMN].fillna(0)
income_qs[MONTHLY_INCOME_COLUMN] = income_qs[MONTHLY_INCOME_COLUMN].replace(999999, 0)
income_qs[income_qs["HOUSE_KEY"]=="202301010101015009017"]


,AÑO,MES,UBIGEO,CONGLOME,VIVIENDA,HOGAR,DOMINIO,ESTRATO,CODPERSO,P530A,P530B,PERIODO,RURAL,HOUSE_KEY,PERSON_KEY


In [4]:
DEMOGRAPHICS_COLUMNS = ["AÑO", "PERIODO", "DOMINIO", "RURAL", "UBIGEO", "CONGLOME"]
aggregations = {c: "first" for c in DEMOGRAPHICS_COLUMNS}
aggregations[MONTHLY_INCOME_COLUMN] = "sum"

# Calculate total income per house ID
df_grouped = income_qs.groupby("HOUSE_KEY").agg(aggregations)
df_grouped['NPER'] = income_qs.groupby("HOUSE_KEY").size()
df_grouped = df_grouped.reset_index()
df_final = compute_poverty_thresholds(df_grouped)
df_final

,HOUSE_KEY,AÑO,PERIODO,DOMINIO,RURAL,UBIGEO,CONGLOME,P530A,NPER,INEI,EXTREME_POVERTY_CUT,POVERTY_CUT,POORNESS
0,20200100500901211,2020,202001,4.0,False,010101,005009,1676.0,2,3,173,316,3
1,20200100500904111,2020,202001,4.0,False,010101,005009,1269.0,2,3,173,316,3
2,20200100500909811,2020,202001,4.0,False,010101,005009,336.0,5,3,173,316,3
3,20200100501500711,2020,202001,4.0,False,010101,005015,1836.0,3,3,173,316,3
4,20200100501501911,2020,202001,4.0,False,010101,005015,0.0,4,3,173,316,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34485,20201202030503111,2020,202012,7.0,False,250301,020305,393.0,1,5,190,332,3
34486,20201202030505211,2020,202012,7.0,False,250301,020305,709.0,1,5,190,332,3
34487,20201202030507311,2020,202012,7.0,False,250301,020305,0.0,1,5,190,332,1
34488,20201202030511611,2020,202012,7.0,False,250301,020305,0.0,2,5,190,332,1


In [5]:
import numpy as np
_, counts = np.unique(df_final["POORNESS"], return_counts=True)
100*counts/len(df_final), len(df_final)

(array([63.88228472,  8.25166715, 27.86604813]), 34490)